In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import json
from underthesea import word_tokenize


import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
device = "cuda" if torch.cuda.is_available() else "cpu"

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


import re

import transformers
from transformers import AutoTokenizer

from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor
from rank_bm25 import BM25Okapi


In [24]:
df = pd.read_json('/media/bbsw/Data1/Hung-ws/lazy/v2/uit-bangb/ise-dsc01-train.json').transpose()
df = df.reset_index(drop=True)

In [25]:
columns = ['context', 'claim', 'verdict', 'evidence']
df = df[columns]

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37967 entries, 0 to 37966
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   context   37967 non-null  object
 1   claim     37967 non-null  object
 2   verdict   37967 non-null  object
 3   evidence  25384 non-null  object
dtypes: object(4)
memory usage: 1.2+ MB


In [27]:
df_evident = df[df['evidence'].notna()]
df_evident = df_evident.reset_index(drop=True)

In [28]:
def handleProcess(context, evident):
    def check_igit(sen):
        igory = ['-', 'com/vocomabelledn\n\nĐình Sơn']
        for ig in igory:
            if ig in sen: return True
        return False
    evident_process = evident.lower().split("\n\n")[-1]
    context_process = context.lower().split("\n\n")[-1]
    if len(evident_process.split())<2 and len(evident)<13:
        return False
    if evident_process==context_process:
        if len(evident_process.split())<20:
            return False
        if len(evident)<100:
            return False
    return True
df_evident_process = df_evident[df_evident.apply(lambda x: handleProcess(x['context'], x['evidence']), axis=1)]
df = df[df.apply(lambda x: x['verdict']=='NEI' or handleProcess(x['context'], x['evidence']), axis=1)]

In [29]:
df.shape

(37571, 4)

In [30]:
df['context'] = df['context'].apply(lambda x: x.lower())
df['claim'] = df['claim'].apply(lambda x: x.lower())

In [32]:
df.head()

,context,claim,verdict,evidence
0,"phát biểu tại lễ ký kết vào ngày 17/11, giám đ...","ngoài việc không giới hạn mức lương, công ty c...",SUPPORTED,"Công ty cũng có chế độ đãi ngộ tốt, có lương t..."
1,"phát biểu tại lễ ký kết vào ngày 17/11, giám đ...","thành lập năm 2016, phát triển phần mềm, tổ ch...",SUPPORTED,"FABA Technology thành lập năm 2016, chuyên cun..."
2,"phát biểu tại lễ ký kết vào ngày 17/11, giám đ...","nhằm phát triển đội ngũ, faba thường tổ chức n...",SUPPORTED,"Bên cạnh đó, FABA thường xuyên tổ chức những d..."
3,"đối với các nhà khoa học, trí thức, điều kiện ...",sở dĩ vị trí kỹ sư phần mềm có mức lương cao n...,SUPPORTED,"Theo Adeco Việt Nam, mức lương cao nhất đối vớ..."
4,"đối với các nhà khoa học, trí thức, điều kiện ...","theo adeco việt nam, mức lương cao nhất đối vớ...",SUPPORTED,"Trong khi đó, theo báo cáo ""Thị trường nhân lự..."


In [33]:
df = df.reset_index(drop=True)
processbar = tqdm(total=len(df)*2)
def tokenizer(text):
    global processbar
    processbar.update(1)
    return word_tokenize(text, format='text')

df['context_tokenizer'] = df['context'].apply(tokenizer)
df['claim_tokenizer'] = df['claim'].apply(tokenizer)
processbar.close()


  0%|          | 0/75142 [00:00<?, ?it/s]

In [34]:
df.shape

(37571, 6)

In [35]:
df

,context,claim,verdict,evidence,context_tokenizer,claim_tokenizer
0,"phát biểu tại lễ ký kết vào ngày 17/11, giám đ...","ngoài việc không giới hạn mức lương, công ty c...",SUPPORTED,"Công ty cũng có chế độ đãi ngộ tốt, có lương t...","phát_biểu tại lễ ký_kết vào ngày 17/11 , giám_...","ngoài việc không giới_hạn_mức lương , công_ty ..."
1,"phát biểu tại lễ ký kết vào ngày 17/11, giám đ...","thành lập năm 2016, phát triển phần mềm, tổ ch...",SUPPORTED,"FABA Technology thành lập năm 2016, chuyên cun...","phát_biểu tại lễ ký_kết vào ngày 17/11 , giám_...","thành_lập năm 2016 , phát_triển phần_mềm , tổ_..."
2,"phát biểu tại lễ ký kết vào ngày 17/11, giám đ...","nhằm phát triển đội ngũ, faba thường tổ chức n...",SUPPORTED,"Bên cạnh đó, FABA thường xuyên tổ chức những d...","phát_biểu tại lễ ký_kết vào ngày 17/11 , giám_...","nhằm phát_triển đội_ngũ , faba thường tổ_chức ..."
3,"đối với các nhà khoa học, trí thức, điều kiện ...",sở dĩ vị trí kỹ sư phần mềm có mức lương cao n...,SUPPORTED,"Theo Adeco Việt Nam, mức lương cao nhất đối vớ...","đối_với các nhà_khoa_học , trí_thức , điều_kiệ...",sở_dĩ vị_trí kỹ_sư phần_mềm có mức lương cao n...
4,"đối với các nhà khoa học, trí thức, điều kiện ...","theo adeco việt nam, mức lương cao nhất đối vớ...",SUPPORTED,"Trong khi đó, theo báo cáo ""Thị trường nhân lự...","đối_với các nhà_khoa_học , trí_thức , điều_kiệ...","theo adeco việt_nam , mức lương cao nhất đối_v..."
...,...,...,...,...,...,...
37566,"""tôi cảm nhận mình đang sống, đang tự do"", bia...",thái lan không dự kiến đón ít nhất 5 triệu khá...,REFUTED,Thái Lan dự kiến đón ít nhất 5 triệu khách Tru...,""" tôi cảm_nhận mình đang sống , đang tự_do "" ,...",thái_lan không dự_kiến đón ít_nhất 5 triệu khá...
37567,"""tôi cảm nhận mình đang sống, đang tự do"", bia...","trước đại dịch, người trung quốc chiếm gần 1/2...",REFUTED,"Trước đại dịch, người Trung Quốc chiếm gần 1/3...",""" tôi cảm_nhận mình đang sống , đang tự_do "" ,...","trước đại_dịch , người trung_quốc chiếm gần 1/..."
37568,"""tôi cảm nhận mình đang sống, đang tự do"", bia...",zhu nói rằng số tiền 290.000 usd không thể mua...,REFUTED,"""Số tiền đó chỉ có thể mua được căn hộ nhỏ ở v...",""" tôi cảm_nhận mình đang sống , đang tự_do "" ,...",zhu nói rằng số tiền 290.000 usd không_thể mua...
37569,"""tôi không có đầu óc đầu tư chứng khoán và đan...","tiền tiết kiệm của các hộ gia đình giảm 9,9 ng...",REFUTED,"Tiền tiết kiệm của các hộ gia đình tăng 9,9 ng...",""" tôi không có đầu_óc đầu_tư chứng_khoán và đa...","tiền tiết_kiệm của các hộ gia_đình giảm 9,9_ng..."


In [38]:
df.to_csv('data/train_v2.csv', index=False, escapechar='\\')

In [ ]:
df.to_json('process.json', orient='records', force_ascii=False, indent=4)

In [ ]:
def find_context(context):
    # Tìm tất cả các số thập phân trong context
    decimal_numbers = re.findall(r'\b\d+\.\d+\b', context)
    return len(decimal_numbers) > 0

In [ ]:
def split_context(context):
    docs = re.split(r'(?<!\d)\.\s*(?![.])', context)
    docs = [doc for doc in docs if doc]
    docs = '\n\n'.join(docs)
    return docs.split('\n\n')

In [ ]:
test = df[df['verdict']=='NEI']

In [ ]:
sample = test.sample()
get_top_rate(sample.claim_tokenizer.values[0], sample.context_tokenizer.values[0])

['" Chứng_kiến giây_phút công_viên Hòa_bình được đổi tên thành_công_viên Hồ_Chí_Minh , dấu_mốc mang tính biểu_tượng về tình_cảm hữu_nghị đặc_biệt Việt_Nam - Cuba , tự đáy lòng mình , chúng_tôi xin bày_tỏ sự xúc_động và biết_ơn chân_thành đối_với tình_cảm và tấm lòng của nhân_dân Cuba anh_em " , Chủ_tịch Quốc_hội Vương_Đình_Huệ phát_biểu tại buổi lễ ',
 'Bức tượng bán_thân làm bằng đồng , cao hai mét , có ghi dòng chữ " Vị anh_hùng giải_phóng dân_tộc và Danh_nhân văn_hóa Việt_Nam " , được đặt ở công_viên Montreau , thành_phố Montreuil , ngoại_ô thủ_đô Paris ',
 'Tượng_đài Chủ_tịch Hồ_Chí_Minh cũng xuất_hiện ở một_số địa_danh khác của Nga như thành_phố Ulianovsk , quê_hương của Lenin , Saint_Petersburg hay Vladivostok ',
 'Công_viên cũng gắn biển cung_cấp thông_tin về thân_thế , sự_nghiệp của Chủ_tịch Hồ_Chí_Minh ',
 'Phía dưới tượng_đài có ghi dòng chữ " Không có gì quý hơn độc_lập tự_do " ',
 'Đến tháng 9/2011 , Ủy_ban Di_sản Quốc_gia Singapore , đại_sứ_quán Việt_Nam tại Singapore và B

In [ ]:
sample.claim.values[0]

'Thơ ca không ca ngợi chủ tịch Hồ Chí Minh, thơ ca ghi lại sự vĩ đại của Người'

In [ ]:
sample.evidence.values

array([None], dtype=object)